In [60]:
import pandas as pd
import sklearn
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import nltk
import re
import numpy as np

from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import LinearSVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv("labeled_data.csv",encoding="latin-1")

In [3]:
def tweet_to_words( raw_tweet ):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_tweet).get_text() 
    # 2. Remove non-letters        
    #letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 3. Convert to lower case, split into individual words
    words = review_text.lower().split()   
    #words = letters_only.lower().split()            
    # 4. In Python, searching a set is much faster than searching
    stops = set(stopwords.words("english")) 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  

In [4]:
num_tweets = data["tweet"].size
clean_train_tweets = []
for i in range( 0, num_tweets ):
    if( (i+1)%10000 == 0 ):
        print ("Tweet %d of %d\n" % ( i+1, num_tweets ))                                                                    
    clean_train_tweets.append( tweet_to_words( data.iloc[i]["tweet"]))

Tweet 10000 of 121054

Tweet 20000 of 121054

Tweet 30000 of 121054

Tweet 40000 of 121054

Tweet 50000 of 121054

Tweet 60000 of 121054



C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://t.co/dALZL6ZuOV" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://t.co/daznQZR8Te" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://t.co/NXgtqSlchG" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

Tweet 70000 of 121054

Tweet 80000 of 121054

Tweet 90000 of 121054

Tweet 100000 of 121054

Tweet 110000 of 121054

Tweet 120000 of 121054



In [29]:
#initialize the count vectorizer
vectorizer = TfidfVectorizer(analyzer = "char",   \
                             ngram_range=(3, 6),  \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 300,  \
                             max_df = 0.85) 
train_data_features = vectorizer.fit_transform(clean_train_tweets)

In [30]:
train_data_features = train_data_features.toarray()
X = pd.DataFrame(train_data_features)
y = data['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69, test_size=0.33)

Logistic Regression

In [39]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
        ('model', LogisticRegression(class_weight='balanced',penalty='l2'))])
param_grid = [{}]
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWa

[CV] ................................................. , total=   3.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWar

[CV] ................................................. , total=   4.5s
[CV]  ................................................................


...

C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.6s
[CV]  ................................................................


C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   4.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   22.5s finished
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [42]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

              precision    recall  f1-score   support

           0       0.71      0.61      0.66      8186
           1       0.90      0.78      0.84     14805
           2       0.77      0.91      0.83     16957

   micro avg       0.80      0.80      0.80     39948
   macro avg       0.79      0.77      0.78     39948
weighted avg       0.81      0.80      0.80     39948



Random Forest

In [45]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
        ('model', RandomForestClassifier(n_estimators=300, random_state=0))])
param_grid = [{}] 
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total= 1.5min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total= 1.4min
[CV]  ................................................................


...

C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total= 1.4min
[CV]  ................................................................


C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total= 1.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  8.1min finished
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [46]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

              precision    recall  f1-score   support

           0       0.77      0.64      0.70      8186
           1       0.88      0.85      0.87     14805
           2       0.80      0.89      0.84     16957

   micro avg       0.82      0.82      0.82     39948
   macro avg       0.82      0.79      0.80     39948
weighted avg       0.82      0.82      0.82     39948



Linear SVC

In [50]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
        ('model', LinearSVC(C=0.05,random_state=0))])

param_grid = [{}] 
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   5.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.7s
[CV]  ................................................................


...

C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   2.9s
[CV]  ................................................................


C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   19.3s finished
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [51]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

              precision    recall  f1-score   support

           0       0.84      0.51      0.64      8186
           1       0.89      0.80      0.84     14805
           2       0.74      0.95      0.83     16957

   micro avg       0.80      0.80      0.80     39948
   macro avg       0.82      0.75      0.77     39948
weighted avg       0.82      0.80      0.79     39948



Extra Trees

In [58]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
        ('model', ExtraTreeClassifier())])

param_grid = [{}]
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   4.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.8s
[CV]  ................................................................


...

C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.5s
[CV]  ................................................................


C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   3.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   20.1s finished
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [59]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

              precision    recall  f1-score   support

           0       0.69      0.64      0.66      8186
           1       0.86      0.84      0.85     14805
           2       0.80      0.84      0.82     16957

   micro avg       0.80      0.80      0.80     39948
   macro avg       0.78      0.77      0.78     39948
weighted avg       0.80      0.80      0.80     39948



Naive Bayes

In [62]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced', penalty="l1", C=0.01))),
        ('model', BernoulliNB())])

param_grid = [{}]
grid_search = GridSearchCV(pipe, param_grid,cv=StratifiedKFold(n_splits=5, random_state=69).split(X_train, y_train), verbose=2)
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   2.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   4.0s
[CV]  ................................................................


...

C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   4.5s
[CV]  ................................................................


C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ................................................. , total=   4.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   19.5s finished
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Kiki\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [63]:
y_preds = model.predict(X_test)
report = classification_report( y_test, y_preds )
print(report)

              precision    recall  f1-score   support

           0       0.80      0.49      0.61      8186
           1       0.82      0.78      0.80     14805
           2       0.72      0.89      0.80     16957

   micro avg       0.77      0.77      0.77     39948
   macro avg       0.78      0.72      0.73     39948
weighted avg       0.77      0.77      0.76     39948

